In [1]:
from py2cytoscape.data.cynetwork import CyNetwork
from py2cytoscape.data.cyrest_client import CyRestClient
from py2cytoscape.data.style import StyleUtil
import py2cytoscape.util.cytoscapejs as cyjs
import py2cytoscape.cytoscapejs as renderer

import networkx as nx
import pandas as pd
import json

<IPython.core.display.Javascript object>

## Basic Setup
* Load a network from URL

In [2]:
cy = CyRestClient()

# Reset session
cy.session.delete()

network1 = cy.network.create_from('http://chianti.ucsd.edu/~kono/data/galFiltered.sif')

## Get Network View Object

In [3]:
view_id_list = network1.get_views()
print(view_id_list)

view1 = network1.get_view(view_id_list[0], format='view')

# This is a CyNetworkView object, not dictionary
print(view1)

view2 = network1.get_view(view_id_list[0], format='json')

# This contains Cytoscape.js style JSON as a Python dictionary
#print(view2)

[4997]


In [4]:
# Get views as Python dictionary
node_views_dict = view1.get_node_views_as_dict() 
edge_views_dict = view1.get_edge_views_as_dict() 

#print(json.dumps(node_views_dict, indent=4))
#print(json.dumps(edge_views_dict, indent=4))

In [5]:
# Get views as NodeView/EdgeView objects
node_views = view1.get_node_views()
nv1 = node_views[0]
nv1_values = nv1.get_values()

#print(json.dumps(nv1_values, indent=4))

# Access single value
nv1_color = nv1.get_value('NODE_FILL_COLOR')
print(nv1_color)

#89D0F5


## Directly set Visual Property values 

In [6]:
node_ids = network1.get_nodes()

# Generate random colors for each node
import random

def generate_random_color(node_ids):
    new_values = {}
    for n in node_ids:
        r = random.randint(0, 255)
        g = random.randint(0, 255)
        b = random.randint(0, 255)
        new_color = '#' + hex(r)[2:] + hex(g)[2:] + hex(b)[2:]
        new_values[n] = new_color
    return new_values

for i in range(1,100):
    view1.update_node_views(visual_property='NODE_FILL_COLOR', values=generate_random_color(node_ids))

## Animation!

In [7]:
def set_value(node_ids, val):
    new_values = {}
    for n in node_ids:
        new_values[n] = val
    return new_values

import math
import time
base_size = 30

for i in range(1,200):
    mult = i * 0.2
    view1.update_node_views(visual_property='NODE_TRANSPARENCY', values=set_value(node_ids, 30+abs(math.sin(mult/math.pi)*255 )))
    view1.update_node_views(visual_property='NODE_WIDTH', values=set_value(node_ids, base_size + math.sin(i/math.pi)*10 ))
    view1.update_node_views(visual_property='NODE_HEIGHT', values=set_value(node_ids, base_size + math.sin(i/math.pi)*10 ))

    time.sleep(0.03)

In [8]:
network_props = view1.get_network_view_as_dict()
print(json.dumps(network_props, indent=4))

{
    "NETWORK_DEPTH": 0.0, 
    "NETWORK_HEIGHT": 400.0, 
    "NETWORK_EDGE_SELECTION": true, 
    "NETWORK_BACKGROUND_PAINT": "#FFFFFF", 
    "NETWORK_CENTER_X_LOCATION": 120.6561279296875, 
    "NETWORK_WIDTH": 550.0, 
    "NETWORK_SIZE": 550.0, 
    "NETWORK_CENTER_Z_LOCATION": 0.0, 
    "NETWORK_SCALE_FACTOR": 1.0, 
    "NETWORK_TITLE": "", 
    "NETWORK_CENTER_Y_LOCATION": -114.6329345703125, 
    "NETWORK_NODE_SELECTION": true
}


In [26]:
from py2cytoscape.data.util_network import NetworkUtil as util

name_map = util.name2suid(network1)
center_node = name_map['YMR043W'] 

node_ids = network1.get_neighbours(center_node)
adj_edges = network1.get_adjacent_edges(center_node)
print(adj_edges)

base_size = 30
for i in range(1, 600):
    mult = i * 0.2
#     view1.update_node_views(visual_property='NODE_TRANSPARENCY', values=set_value(node_ids, 30+abs(math.sin(mult/math.pi)*255 )))
    view1.update_node_views(visual_property='NODE_WIDTH', values=set_value(node_ids, base_size + math.sin(i/math.pi)*10 ))
    view1.update_node_views(visual_property='NODE_HEIGHT', values=set_value(node_ids, base_size + math.sin(i/math.pi)*10 ))
    # FIX CYREST!!!!
    view1.update_edge_views(visual_property='EDGE_STROKE_UNSELECTED_PAINT', values=set_value(adj_edges, 'red'))
    time.sleep(0.05)

[4544, 4542, 4540, 4538, 4536, 4534, 4532, 4530, 4910, 4897, 4847, 4799, 4752, 4739, 4737, 4689, 4571, 4562]
